<a href="https://colab.research.google.com/github/bibikkkka/Big-Data/blob/LR0/6405_%D0%A4%D0%B8%D0%BB%D0%B8%D0%BF%D0%BF%D0%BE%D0%B2%D0%90%D0%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение в MapReduce модель на Python


In [98]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [99]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [100]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [101]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [102]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [103]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [104]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [105]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [106]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [107]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [108]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [109]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [110]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [111]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [112]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.13255473563838),
 (1, 2.13255473563838),
 (2, 2.13255473563838),
 (3, 2.13255473563838),
 (4, 2.13255473563838)]

## Inverted index

In [113]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [114]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [115]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [116]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('it', 18)]),
 (1, [('banana', 2), ('is', 18), ('what', 10)])]

## TeraSort

In [117]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.006877044737948479),
   (None, 0.008650092026038059),
   (None, 0.04203806180490677),
   (None, 0.06191813041258909),
   (None, 0.0787842274057522),
   (None, 0.1393219209895623),
   (None, 0.15343311246013447),
   (None, 0.2735477877527218),
   (None, 0.2807077601406802),
   (None, 0.28746675474605854),
   (None, 0.30368760364774905),
   (None, 0.30426432175915863),
   (None, 0.34142282446968764),
   (None, 0.39259913148740966),
   (None, 0.41218475742467175),
   (None, 0.4885722118412802),
   (None, 0.4957536507831083)]),
 (1,
  [(None, 0.5038145775244703),
   (None, 0.5166030916364686),
   (None, 0.688310694533284),
   (None, 0.6894692854287802),
   (None, 0.7322857321760471),
   (None, 0.7498625941232887),
   (None, 0.7582622485128527),
   (None, 0.7640914215130273),
   (None, 0.7816051090593832),
   (None, 0.7998620094913161),
   (None, 0.842000662729942),
   (None, 0.9430906818824387),
   (None, 0.9454204584189059)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [118]:
from typing import Iterator
from typing import NamedTuple # requires python 3.6+
import numpy as np

numbers = np.random.randint(1, 100, size=10)
print(numbers)

def MAP(id, n):
  yield (id, n)

def REDUCE(key:int, numbers:list):
  yield (None, max(numbers))

def RECORDREADER():
  for n in numbers:
    yield (None, n)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output



[17 28 72 32 40 35  1 51  7 78]


[(None, 78)]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [119]:
from typing import Iterator
from typing import NamedTuple # requires python 3.6+
import numpy as np

numbers = np.random.randint(1, 100, size=10)
print(numbers)

def MAP(id, n):
  yield (id, n)

def REDUCE(key:int, numbers:list):
  sum = 0
  count = 0
  for n in numbers:
    sum += n
    count += 1
  if (count > 0):
    yield(None, sum/count)
  else:
    yield (None, 0)

def RECORDREADER():
  for n in numbers:
    yield (None, n)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[23 52 59 55 35 33 32 24  4 72]


[(None, 38.9)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [120]:
from typing import Iterator
from typing import NamedTuple # requires python 3.6+
import numpy as np
from collections import defaultdict

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def groupByKey(iterable):
  groups = defaultdict(list)
  for key, value in sorted(iterable, key=lambda x: x[0]):
    groups[key].append(value)
  return groups.items()

def RECORDREADER(key:str):
   for u in input_collection:
      yield getattr(u, key), u

keys = ["id", "gender", "age"]
for key in keys:
  print(f"Grouping by {key}:")
  print(list(groupByKey(RECORDREADER(key))))
  print("\n")

Grouping by id:
[(0, [User(id=0, age=55, social_contacts=20, gender='male')]), (1, [User(id=1, age=25, social_contacts=240, gender='female')]), (2, [User(id=2, age=25, social_contacts=500, gender='female')]), (3, [User(id=3, age=33, social_contacts=800, gender='female')])]


Grouping by gender:
[('female', [User(id=1, age=25, social_contacts=240, gender='female'), User(id=2, age=25, social_contacts=500, gender='female'), User(id=3, age=33, social_contacts=800, gender='female')]), ('male', [User(id=0, age=55, social_contacts=20, gender='male')])]


Grouping by age:
[(25, [User(id=1, age=25, social_contacts=240, gender='female'), User(id=2, age=25, social_contacts=500, gender='female')]), (33, [User(id=3, age=33, social_contacts=800, gender='female')]), (55, [User(id=0, age=55, social_contacts=20, gender='male')])]




### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [121]:
from os import name
from typing import Iterator
import numpy as np

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for u in split:
      yield (u.id, u)

  split_size = int(np.ceil(len(input_collection)/maps))
  for i in range(0, len(input_collection), split_size):
    yield RECORDREADER(input_collection[i:i+split_size])

def MAP(id:int, user:User):
  yield (id, user)

def REDUCE(id:int, users: list) -> Iterator[tuple]:
  first_user = users[0]
  yield (id, first_user)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

5 key-value pairs were sent over a network.


[(0,
  [(0, User(id=0, age=55, social_contacts=20, gender='male')),
   (2, User(id=2, age=25, social_contacts=500, gender='female'))]),
 (1,
  [(1, User(id=1, age=25, social_contacts=240, gender='female')),
   (3, User(id=3, age=33, social_contacts=800, gender='female'))])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [122]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, u:User):
  if (u.gender == 'female'):
    yield (u, u)

def REDUCE(_, user:User):
  yield user

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[[User(id=1, age=25, social_contacts=240, gender='female')],
 [User(id=2, age=25, social_contacts=500, gender='female')],
 [User(id=3, age=33, social_contacts=800, gender='female')]]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [123]:
class Projection(NamedTuple):
  id: int
  age: int
  social_contacts: int


data_list = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=240),
    User(id=3, age=33, gender='female', social_contacts=800)
]


def RECORDREADER():
  for u in data_list:
    yield (u.id, u)


def MAP(id:int, u:User):
  new_user = Projection(id, u.age, u.social_contacts)
  yield(new_user, new_user)


def REDUCE(p:Projection, usersList:list):
  yield (p, p)


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(Projection(id=0, age=55, social_contacts=20),
  Projection(id=0, age=55, social_contacts=20)),
 (Projection(id=1, age=25, social_contacts=240),
  Projection(id=1, age=25, social_contacts=240)),
 (Projection(id=2, age=25, social_contacts=240),
  Projection(id=2, age=25, social_contacts=240)),
 (Projection(id=3, age=33, social_contacts=800),
  Projection(id=3, age=33, social_contacts=800))]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [124]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

first_input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

second_input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=10, age=25, gender='female', social_contacts=240),
    User(id=20, age=25, gender='female', social_contacts=500),
    User(id=30, age=33, gender='female', social_contacts=800)
]

union_input_collection = [first_input_collection, second_input_collection]

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def MAP(id:int, u:User):
  yield (u, u)

def REDUCE(u:User, users:list):
  yield (u, u)

def RECORDREADER():
  flattened_data = flatten([first_input_collection, second_input_collection])
  for u in flattened_data:
      yield (u.id, u)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=2, age=25, social_contacts=500, gender='female'),
  User(id=2, age=25, social_contacts=500, gender='female')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female')),
 (User(id=10, age=25, social_contacts=240, gender='female'),
  User(id=10, age=25, social_contacts=240, gender='female')),
 (User(id=20, age=25, social_contacts=500, gender='female'),
  User(id=20, age=25, social_contacts=500, gender='female')),
 (User(id=30, age=33, social_contacts=800, gender='female'),
  User(id=30, age=33, social_contacts=800, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [125]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

first_input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

second_input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=10, age=25, gender='female', social_contacts=240),
    User(id=20, age=25, gender='female', social_contacts=500),
    User(id=30, age=33, gender='female', social_contacts=800)
]

union_input_collection = [first_input_collection, second_input_collection]

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def MAP(id:int, u:User):
  yield (u, u)

def REDUCE(u:User, users:list):
  if (len(users) == 2 and users[0] == users[1]):
    yield (u, u)

def RECORDREADER():
  flattened_data = flatten([first_input_collection, second_input_collection])
  for u in flattened_data:
      yield (u.id, u)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [126]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

first_input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

second_input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=10, age=25, gender='female', social_contacts=240),
    User(id=20, age=25, gender='female', social_contacts=500),
    User(id=30, age=33, gender='female', social_contacts=800)
]

union_input_collection = [first_input_collection, second_input_collection]

def MAP(id:int, u:User):
  yield (u, id)

def REDUCE(u:User, users:list):
  if (len(users) == 1 and users[0] == 0):
    yield (u, u)

def RECORDREADER():
  for i, users in enumerate(union_input_collection):
    for u in users:
      yield (i, u)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=2, age=25, social_contacts=500, gender='female'),
  User(id=2, age=25, social_contacts=500, gender='female')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female'))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [127]:
from typing import NamedTuple, List, Tuple
from itertools import chain

class AgeToSocialContacts(NamedTuple):
    age: int
    social_contacts: int

class Projection(NamedTuple):
    id: int
    age: int

input_collection = [
    Projection(id=0, age=55),
    Projection(id=1, age=25),
    Projection(id=2, age=25),
    Projection(id=3, age=33)
]

age_to_social_contacts_collection = [
    AgeToSocialContacts(age=55, social_contacts=20),
    AgeToSocialContacts(age=25, social_contacts=240),
    AgeToSocialContacts(age=33, social_contacts=800)
]

data = [input_collection, age_to_social_contacts_collection]

def RECORDREADER():
  for i, users in enumerate(data):
    for u in users:
      yield (i, u)

def MAP(id, user):
    if id==0:
        yield (user.age, (0, user.id))
    else:
        yield (user.age, (1, user.social_contacts))

def REDUCE(key, records):
    data_R = [a for (setId, a) in records if setId == 0]
    data_S = [c for (setId, c) in records if setId == 1]

    for a in data_R:
      for c in data_S:
        yield (a, key, c)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 55, 20), (1, 25, 240), (2, 25, 240), (3, 33, 800)]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [128]:
from typing import NamedTuple, Iterator

class AgeToSocialContacts(NamedTuple):
    age: int
    social_contacts: int

class Projection(NamedTuple):
    id: int
    age: int

input_collection = [
    Projection(id=0, age=55),
    Projection(id=1, age=25),
    Projection(id=2, age=25),
    Projection(id=3, age=33)
]

age_to_social_contacts_collection = [
    AgeToSocialContacts(age=55, social_contacts=20),
    AgeToSocialContacts(age=25, social_contacts=240),
    AgeToSocialContacts(age=33, social_contacts=800)
]

def MAP(id: int, projection: Projection):
    yield (projection.age, id)

def REDUCE(key: int, values: Iterator[int]):
    total_ids = sum(values)
    yield (key, total_ids)

def RECORDREADER():
    for projection in input_collection:
        yield projection.id, projection

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(output)

[(55, 0), (25, 3), (33, 3)]


#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [129]:
import numpy as np

mat = np.ones((5, 5))
vec = np.array([2, 5, 1, 1, 1])

maps = 3
reducers = 1

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for i in range(split.shape[0]):
            for j in range(split.shape[1]):
                yield ((i, j), (split[i, j], vec[j], split.shape[1]))

    split_size = int(np.ceil(len(mat) / maps))

    for i in range(0, len(mat), split_size):
        yield RECORDREADER(mat[i: i + split_size])

def MAP(coordinates:(int, int), values:(int, int, int)):
    i, j = coordinates
    mat_val, vec_val, cols = values
    yield ((i, cols), mat_val * vec_val)

def REDUCE(keys:(int, int), products:Iterator[NamedTuple]):
    i, cols = keys
    yield (i, sum(products) / (len(products) / cols))

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

25 key-value pairs were sent over a network.


[(0, [(0, 10.0), (1, 10.0)])]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [130]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [131]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
    (j, k) = k1
    w = v1
    for i in range(small_mat.shape[0]):
        yield ((i, k), small_mat[i, j] * w)

def REDUCE(key, values):
    (i, k) = key
    yield (key, sum(values))

Проверьте своё решение

In [132]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [133]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [134]:
def RECORDREADER():
  for k in range(big_mat.shape[1]):
    for i in range(small_mat.shape[0]):
      for j in range(small_mat.shape[1]):
        yield ((i, k), (0, small_mat[i, j]))

  for i in range(small_mat.shape[0]):
    for j in range(big_mat.shape[0]):
      for k in range(big_mat.shape[1]):
        yield ((i, k), (1, big_mat[j, k]))


def MAP(coordinates:(int, int), values:(int, int)):
  yield (coordinates, values)


def REDUCE(coordinates:(int, int), values:list):
  small_mat_vals = [val for mat_idx, val in values if mat_idx == 0]
  big_mat_vals = [val for mat_idx, val in values if mat_idx == 1]
  yield (coordinates, np.dot(small_mat_vals, big_mat_vals))


output = MapReduce(RECORDREADER, MAP, REDUCE)

np.allclose(reference_solution, asmatrix(output)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [135]:
maps = 2
reducers = 1

def INPUTFORMAT():
  global maps
  small_rows = small_mat.shape[0]
  big_rows = big_mat.shape[0]


  def RECORDREADER_SMALL(split, idx):
    for k in range(big_mat.shape[1]):
      for i in range(split.shape[0]):
        for j in range(split.shape[1]):
          yield ((i + idx, k), (0, split[i, j]))


  split_size = int(np.ceil(small_rows / maps))
  for i in range(0, small_rows, split_size):
    yield RECORDREADER_SMALL(small_mat[i : i + split_size], i)


  def RECORDREADER_BIG(split):
    for i in range(small_rows):
      for j in range(split.shape[0]):
        for k in range(split.shape[1]):
          yield ((i, k), (1, split[j, k]))


  split_size = int(np.ceil(big_rows / maps))
  for i in range(0, big_rows, split_size):
    yield RECORDREADER_BIG(big_mat[i : i + split_size])


def MAP(coordinates:(int, int), values:(int, int)):
  yield (coordinates, values)


def REDUCE(coordinates:(int, int), values:list):
  small_mat_vals = [val for mat_idx, val in values if mat_idx == 0]
  big_mat_vals = [val for mat_idx, val in values if mat_idx == 1]
  yield (coordinates, np.dot(small_mat_vals, big_mat_vals))


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

480 key-value pairs were sent over a network.


In [136]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [137]:
maps = 2
reducers = 1


def INPUTFORMAT():
  global maps
  small_rows = small_mat.shape[0]
  small_cols = small_mat.shape[1]

  big_rows = big_mat.shape[0]
  big_cols = big_mat.shape[1]


  def FIRST_RECORDREADER_SMALL(split, idx):
    for k in range(big_cols):
      for i in range(split.shape[0]):
        for j in range(split.shape[1]):
          yield ((i + idx, k), (0, split[i, j]))


  def SECOND_RECORDREADER_SMALL(split, idx):
    for k in range(big_cols):
      for i in range(split.shape[0]):
        for j in range(split.shape[1]):
          yield ((i + idx, k), (0, split[i, j]))


  def FIRST_RECORDREADER_BIG(split, idx):
    for i in range(small_rows):
      for j in range(split.shape[0]):
        for k in range(split.shape[1]):
          yield ((i, k + idx), (1, split[j, k]))


  def SECOND_RECORDREADER_BIG(split, idx):
    for i in range(small_rows):
      for j in range(split.shape[0]):
        for k in range(split.shape[1]):
           yield ((i, k + idx), (1, split[j, k]))


  small_readers = {
      0: FIRST_RECORDREADER_SMALL,
      1: SECOND_RECORDREADER_SMALL
  }


  big_readers = {
       0: FIRST_RECORDREADER_BIG,
       1: SECOND_RECORDREADER_BIG
  }

  split_size_rows = int(np.ceil(small_rows / maps))
  split_size_cols = int(np.ceil(small_cols / maps))

  counter = -1
  for i in range(0, small_rows, split_size_rows):
    for j in range(0, small_cols, split_size_cols):
      counter += 1
      yield small_readers[counter % len(small_readers)](small_mat[i : i + split_size_rows, j : j + split_size_cols], i)


  split_size_rows = int(np.ceil(big_rows / maps))
  split_size_cols = int(np.ceil(big_cols / maps))

  counter = -1
  for i in range(0, big_rows, split_size_rows):
    for j in range(0, big_cols, split_size_cols):
      counter += 1
      yield big_readers[counter % len(big_readers)](big_mat[i : i + split_size_rows, j : j + split_size_cols], j)


def MAP(coordinates:(int, int), values:(int, int)):
  yield (coordinates, values)


def REDUCE(coordinates:(int, int), values:list):
  small_mat_vals = [val for mat_idx, val in values if mat_idx == 0]
  big_mat_vals = [val for mat_idx, val in values if mat_idx == 1]
  yield (coordinates, np.dot(small_mat_vals, big_mat_vals))


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

480 key-value pairs were sent over a network.


In [138]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True